In [3]:

import numpy as np
from sklearn.metrics import RocCurveDisplay
from drowsiness_detection import config
from dataclasses import dataclass
from drowsiness_detection.data import load_preprocessed_train_val_test_splits, session_type_mapping, load_nn_data

In [4]:
prepr_path = config.SOURCES_ROOT_PATH.parent.joinpath("data/preprocessed/60sec")

In [5]:


# data = load_engineered_60_sec_data()

In [6]:
# X_train, X_test, y_train, y_test, _ = load_preprocessed_train_test_splits(
#     data_path=config.PATHS.WINDOW_FEATURES,
#     exclude_sess_type=session_type_mapping["a"],
#     num_targets=2,
#     seed=42,
#     test_size=.2,
#     split_by_subjects=True,
# )
prepr_path = config.SOURCES_ROOT_PATH.parent.joinpath("data/preprocessed/60sec")
prepr_path.mkdir(exist_ok=1)

prepr_path

PosixPath('/home/tim/IM/data/preprocessed/60sec')

In [41]:
seed = 44
num_fold = 3


In [42]:
config.set_paths(30, 60)

In [43]:
X_train, X_val, X_test, y_train, y_val, y_test, (
train_ids, val_ids, test_ids) = load_preprocessed_train_val_test_splits(
    data_path=config.PATHS.WINDOW_FEATURES,
    exclude_sess_type=session_type_mapping["a"],
    num_targets=2,
    seed=seed,
    test_size=.2,
    split_by_subjects=True,
)

In [44]:
print(train_ids, val_ids, test_ids)
# fold 1: [31 39 46 23 47 16 45  6 29 43 22 21 48  3 42 10 15  8 19 40 24  2 26 36
# 11 17 33  4] [ 1  5 14 20 12 32 44 52 53] [27 28 41 13  9  7 38 18 50]

# fold 2: [15  2  1 20  6  5 22  9 24 41 18  3 27 32  4 10 50 13 11 47 48 29 40 45
# 39] [33 52 17 12 28 53 16 23 44 36] [38 21 46  7 19  8 14 26 31 43 42]

# fold 3: [27 50 31 46 33 20 12 22 24 41 42 44 52  8  5  1 15 45 48 40 11 43 23 26
# 21 32  9] [19  2 39 29 38 47 53 10  4  7] [ 6 16 36 28 18 14 13  3 17]

[27 50 31 46 33 20 12 22 24 41 42 44 52  8  5  1 15 45 48 40 11 43 23 26
 21 32  9] [19  2 39 29 38 47 53 10  4  7] [ 6 16 36 28 18 14 13  3 17]


In [45]:

import random
from drowsiness_detection.data import  get_feature_data, preprocess_feature_data
exclude_sess_type = session_type_mapping["a"]
data_path = config.PATHS.WINDOW_FEATURES
num_targets = 2

np.random.seed(seed)
random.seed(seed)

data = get_feature_data(data_path=data_path)
X, y, subject_data = preprocess_feature_data(feature_data=data,
                                             exclude_sess_type=exclude_sess_type,
                                             num_targets=num_targets)
# subject data is sess_type + subject id

In [46]:
print(f"{len(X_train)=}")
print(f"{len(X_val)=}")
print(f"{len(X_test)=}")


len(X_train)=18828
len(X_val)=6490
len(X_test)=6818


In [47]:
print(f"{len(subject_data[np.isin(subject_data[:,1], train_ids)])=}")
print(f"{len(subject_data[np.isin(subject_data[:,1], val_ids)])=}")
print(f"{len(subject_data[np.isin(subject_data[:,1], test_ids)])=}")


len(subject_data[np.isin(subject_data[:,1], train_ids)])=18828
len(subject_data[np.isin(subject_data[:,1], val_ids)])=6490
len(subject_data[np.isin(subject_data[:,1], test_ids)])=6818


In [48]:
engineered_feature_data_path = prepr_path.joinpath("engineered_features")
engineered_feature_data_path.mkdir(exist_ok=1)

In [49]:
np.save(str(engineered_feature_data_path.joinpath(f"X_train_{num_fold}.npy")), X_train)
np.save(str(engineered_feature_data_path.joinpath(f"X_val_{num_fold}.npy")), X_val)
np.save(str(engineered_feature_data_path.joinpath(f"X_test_{num_fold}.npy")), X_test)

np.save(str(engineered_feature_data_path.joinpath(f"y_train_{num_fold}.npy")), y_train)
np.save(str(engineered_feature_data_path.joinpath(f"y_val_{num_fold}.npy")), y_val)
np.save(str(engineered_feature_data_path.joinpath(f"y_test_{num_fold}.npy")), y_test)

In [50]:
raw_feature_data_path = prepr_path.joinpath("raw_features")
raw_feature_data_path.mkdir(exist_ok=1)

In [51]:
X,y, subject_data = load_nn_data(data_path=config.PATHS.WINDOW_DATA)

In [52]:
train_mask = np.isin(subject_data[:,1],train_ids)
val_mask = np.isin(subject_data[:,1],val_ids)
test_mask = np.isin(subject_data[:,1],test_ids)


In [53]:
X_train = X[train_mask]
X_val = X[val_mask]
X_test = X[test_mask]

y_train = y[train_mask]
y_val = y[val_mask]
y_test = y[test_mask]

In [54]:
np.save(str(raw_feature_data_path.joinpath(f"X_train_{num_fold}.npy")), X_train)
np.save(str(raw_feature_data_path.joinpath(f"X_val_{num_fold}.npy")), X_val)
np.save(str(raw_feature_data_path.joinpath(f"X_test_{num_fold}.npy")), X_test)

np.save(str(raw_feature_data_path.joinpath(f"y_train_{num_fold}.npy")), y_train)
np.save(str(raw_feature_data_path.joinpath(f"y_val_{num_fold}.npy")), y_val)
np.save(str(raw_feature_data_path.joinpath(f"y_test_{num_fold}.npy")), y_test)